# OFRAK Debate

**First**, Why.

OFRAK, the Open Firmware Reverse Analysis Konsole, is a framework that lets you do very fancy firmware analysis, very brave assured micro firmware patching, in a simple interface that lets you roll in all your favorite tools into an eloquent, agile ninja workflow. This is not a tool to run a LLM-driven fevor dream of a presidential debate.


**Second**, Huh?
Yes. OFRAK is kick ass for firmware stuff. But can you do fancy complicated things that isn't in the old boring process of unpack, analyze, modify, repack? Let's find out.

**Third**, How?
Look below. You can do intersting scaled things if you know how to work with FirmwareObject (<-replace with what we call it now), our new dumb simple LLM analyzer interface, and a little healthy sense of amazifunventure.

**Fourth**, Who the f#@(@#! is this clown moderator?
IT is a doctor, anonymous user. And that's Doctor Clown to you, bachelor.

Now let's get on with the show. Install OFRAK, Pull OFRAK project, put in your own API tokens for the LLM stuff, laugh or sigh. up to you -)

## Install
OFRAK's LLM featuers are on a wip dev branch. Let's pull in that version here and get that installed.

In [ ]:
!git clone -b feature/llm-analyzer https://github.com/rbs-jacob/ofrak.git

In [ ]:
!pushd ofrak/ofrak_type && make install && popd
!pushd ofrak/ofrak_io && make install && popd
!pushd ofrak/ofrak_patch_maker && make install && popd
!pushd ofrak/frontend && npm install && npm run build && popd
!pushd ofrak/ofrak_core && cp -r ../frontend/dist ofrak/gui/public && make install && pushd

If that all worke, you should see some Llm analyzers:

In [11]:
!ofrak list | grep Llm

	LlmAnalyzer
	LlmFunctionAnalyzer
	LlmProgramAnalyzer


We'll also need to install F5-TTS

In [ ]:
!python -m pip install pydub~=0.25.1 git+https://github.com/SWivid/F5-TTS.git

## Run the code!

In [1]:
# Set your TTS server URL here
TTS_URL = "http://0.0.0.0:7860"

In [2]:
import IPython

In [5]:
from io import BytesIO

from ofrak import OFRAK
from ofrak.core import *
from ofrak.core.llm import *

from debate_components import *

In [6]:
o = OFRAK()
import debate_components

o.discover(debate_components)
ofrak_context = await o.create_ofrak_context()

Using OFRAK Pro License: 06f899eda0cb58e3729a2296ce5fe994.
Run 'ofrak license' to see full license details.


In [7]:
dr_ang_cui_resource = await ofrak_context.create_root_resource_from_file(
    "questions.txt",
)
dr_ang_cui_resource.add_view(Person("Dr. Ang Cui", "voices/Ang_Cui.wav"))
await dr_ang_cui_resource.save()

djt_resource = await ofrak_context.create_root_resource(
    name="Donald Trump", data=b"", tags=(GenericBinary, Person)
)
djt_resource.add_view(Person("Donald Trump", "voices/Donald_Trump.wav"))
await djt_resource.save()

kh_resource = await ofrak_context.create_root_resource(
    name="Kamala Harris",
    data=b"",
    tags=(GenericBinary, Person),
)
kh_resource.add_view(Person("Kamala Harris", "voices/Kamala_Harris.wav"))
await kh_resource.save()

In [10]:
debate_questions = await dr_ang_cui_resource.get_data()
print(debate_questions.decode())
questions = enumerate(debate_questions.decode().splitlines())

What is your plan to address the rising national debt and reduce it within the next decade?
How do you intend to balance economic growth with increasing income inequality in the United States?
What steps would you take to strengthen America's alliances and partnerships in a rapidly changing global landscape?
How do you propose addressing climate change, particularly in regards to reducing carbon emissions and transitioning to renewable energy sources?
What is your stance on gun control and would you support stricter regulations in the wake of mass shootings?
How would you reform the current healthcare system to make it more affordable and accessible to all Americans?
What role would you give the federal government in addressing issues related to social media regulation and online misinformation?
How do you plan to invest in education, particularly in regards to early childhood education and vocational training programs?
Would you support increasing taxes on corporations to fund infrast

In [15]:
async def ask_question(
    question: str, moderator: Resource, candidate_one: Resource, candidate_two: Resource
):
    i, question = question
    print(f"[+] Asking question {i + 1}")
    await dr_ang_cui_resource.run(
        TTSAnalyzer,
        TTSAnalyzerConfig(start_line=i, num_lines=i + 1, client_url=TTS_URL),
    )

    IPython.display.Audio(
        dr_ang_cui_resource.get_attributes(WavFile).wav, autoplay=True
    )

    IPython.display.display(
        IPython.display.Audio(
            dr_ang_cui_resource.get_attributes(WavFile).wav, autoplay=True
        )
    )

    await djt_resource.run(
        AnswerQuestionModifier, AnswerQuestionConfig(question, tts_client_url=TTS_URL)
    )
    IPython.display.display(
        IPython.display.Audio(djt_resource.get_attributes(WavFile).wav, autoplay=True)
    )

    await kh_resource.run(
        AnswerQuestionModifier, AnswerQuestionConfig(question, tts_client_url=TTS_URL)
    )
    IPython.display.display(
        IPython.display.Audio(kh_resource.get_attributes(WavFile).wav, autoplay=True)
    )

In [ ]:
await ask_question(next(questions), dr_ang_cui_resource, djt_resource, kh_resource)

In [ ]:
await ask_question(next(questions), dr_ang_cui_resource, djt_resource, kh_resource)